In [ ]:
import pandas as pd

In [ ]:
all_features = ["text_ tokens"]

test = pd.read_csv("s3://recsys-challenge-2020/competition_test.tsv", encoding="utf-8",
                    names = all_features, usecols= [0], sep="\x01")

In [ ]:
from transformers import *

In [ ]:
test_text = pd.DataFrame()

In [ ]:
def calculate_text(row):
    tweet_tokens = tokenizer.decode(list(map(int, row.split('\t'))))
    return tweet_tokens

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [ ]:
test_text['user_text'] = test['text_ tokens'].apply(lambda x: calculate_text(x))

In [ ]:
test_text.to_csv('s3://recsys-challenge-2020/test_set_text.csv', index = False)

In [ ]:
cleaned_text_test = pd.DataFrame()

In [ ]:
def tokenize_text(row, index):
    if index % 100000 == 0:
        print(index)
    row1 = row.replace('[CLS]','')
    row2 = row1.replace('[SEP]', '')
    result = re.sub(r"http\S+", "", row2)
    tokenizer = RegexpTokenizer(r'\w+')
    token_list = tokenizer.tokenize(result)
    feature =  ' '.join([w.lower() for w in token_list])
    return feature

In [ ]:
cleaned_text_test['tweet_text'] = test_text.apply(lambda x: tokenize_text(x.user_text, x.name), axis = 1)

In [ ]:
all_features = ["language"]
test = pd.read_csv("s3://recsys-challenge-2020/competition_test.tsv", encoding="utf-8",
                    names = all_features, usecols= [7], sep="\x01"
                   )
print('test language loaded')

In [ ]:
language_code_name_pairs = pd.read_csv('s3://recsys-challenge-2020/language_code_name_pairs.csv')

In [ ]:
test_input = pd.merge(test, language_code_name_pairs, how = 'inner',\
left_on = 'language', right_on = 'code')[['name']]